<a href="https://colab.research.google.com/github/ricotta-jpgomes/yt_datatools/blob/main/ytRelated_extractor_v1/getEdges_module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get edges: extraindo as coordenadas da nossa rede de vídeos recomendados



Este notebook consiste em uma sequência de código para extrair vídeos relacionados a um vídeo seed, determinado pelo usuário, e gerar um arquivo .csv do tipo source - target, representando as coordenadas da nossa rede, ou seja, como os vídeos se recomendam. 

Nesta etapa, eu esgoto o limite diário da API 😰, e por conta disso implementei um log simples em um arquivo .txt para proceder o resto da extração no dia seguinte, quando a cota é restaurada (o código onde eu obtenho os detalhes dos vídeos capturados nessa primeira etapa pode ser encontrado [aqui](https://colab.research.google.com/drive/1DvzGCuXJbrrdqLAvZJURNPkC9gV3LwUm?usp=sharing). Deste modo, consigo obter uma profundidade maior na minha rede, ao realizar mais iterações sobre os vídeos recomendados (recomendados dos recomendados e por aí vai). A quantidade máxima de iterações obtidas com esse notebook, até o limite diário da API, é de 2, para 10 recomendados por vídeo desde a seed.  


##1. Instalando Dependências

In [ ]:
# cliente google
!pip install --upgrade google-api-python-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.7 MB 4.2 MB/s 
  Attempting uninstall: google-auth-httplib2
    Found existing installation: google-auth-httplib2 0.0.4
    Uninstalling google-auth-httplib2-0.0.4:
      Successfully uninstalled google-auth-httplib2-0.0.4
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.11
    Uninstalling google-api-python-client-1.12.11:
      Successfully uninstalled google-api-python-client-1.12.11


In [ ]:
# dotenv
! pip install python_dotenv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**1.1. Conectando o Google Drive**

A primeira importação vai ser do módulo que nos permite acessar nosso drive. Esse módulo é extremamente importante pois com ele posso resgatar minha chave de api, bem como abrir e manipular os arquivos que serão salvos em uma pasta que no meu Drive eu dedico exclusivamente a datasets. Você pode especificar o caminho que quiser dentro do seu drive pessoal para esses arquivos. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##2. Resgatando funções predefinidas

In [ ]:
import sys # biblioteca para adicionar funções definidas em arquivo separado

In [ ]:
sys.path.insert(0, '/content/drive/MyDrive/Colab Notebooks/youTube/ytRelated_extractor_v1/helpers')

In [ ]:
from general import get_extraction_date
from youtube import get_video_id, basic_search, deep_search

##4. Requisição e processamento dos dados

Começamos aqui definindo nosso vídeo seed, a partir do qual vamos montar nossa rede de recomendações. Extraímos o id desse vídeo para realizar a rquisição inicial.

In [ ]:
link = input()
video_id = get_video_id(link)

https://www.youtube.com/watch?v=XCKPLBi9bmE


In [ ]:
video_id # Tudo certo!

'XCKPLBi9bmE'

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/youTube/ytRelated_extractor_v1/log.txt', 'w') as f:
  f.write(f'{video_id}\n')

**Realizando as requisições: controle de iterações e persistência dos dados**


Vou definir aqui o caminho para o arquivo que será manipulado ao longo das requisições: 

In [ ]:
# Definindo caminho para arquivo que receberá os dados das requisições
file_name = input('digite o nome do arquivo: ') + get_extraction_date()[1]
path = f'/content/drive/MyDrive/Datasets/network_files/{file_name}.csv'

digite o nome do arquivo: teste


In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/youTube/ytRelated_extractor_v1/log.txt', 'a') as f:
  f.write(path)

Agora sim começamos com as requisições. Inicializo a variável loop em zero e faço a primeira requisição com a id seed obtida anteriormente. No primeiro loop, crio o arquivo, abrindo em modo 'write', e armazeno os dados obtidos, fechando o arquivo em seguida.

In [ ]:
# Primeiro loop
loop = 0 
coords, next_loop_ids = basic_search(video_id)    
    
with open(path, 'w') as f:
  writer = csv.DictWriter(f, fieldnames=['source', 'target', 'type'])
  writer.writeheader()
  
  for source_id, target_id in zip(coords['source'], coords['target']):
      row = {'source' : source_id, 'target' : target_id, 'type' : 'Direct'}  
      writer.writerow(row)

extracting coords for video XCKPLBi9bmE...
ok!


O controle das iterações vai ser feito utilizando a constante MAX_DEPTH e a variável booleana next_loop. Por conta das limitações da chave de api, MAX_DEPTH vai ser definida com o valor de 2, ou seja, duas iterações a partir da requisição inicial. A cada loop, os dados serão armazenados em arquivo para evitar sobrecarga da RAM ou perda de dados caso ocorra algum erro durante as requisições.

In [ ]:
MAX_DEPTH = 2
next_loop = True

In [ ]:
# Demais loops
while next_loop:
    loop += 1
    print(f'Entering loop {loop}:')

    loop_data = deep_search(next_loop_ids.copy())
    next_loop_ids.clear()

    with open(path, 'a') as f:
      writer = csv.DictWriter(f, fieldnames=['source', 'target', 'type']) # abro o arquivo em modo 'a' a cada novo loop

      for data in loop_data:  
        coords = data[0]
        next_loop_ids += data[1]
        
        for source_id, target_id in zip(coords['source'], coords['target']):
          row = {'source' : source_id, 'target' : target_id, 'type' : 'Direct'}  
          writer.writerow(row)

    if loop == MAX_DEPTH:
      next_loop = False   

Entering loop 1:
extracting coords for video p4xRayAhAQs...
ok!
extracting coords for video nANSvLKRGTk...
ok!
extracting coords for video mRMGg04r_Bw...
ok!
Entering loop 2:
extracting coords for video OHCc9-ygxPc...
ok!
extracting coords for video k1IzeKB0Ceg...
ok!
extracting coords for video haMaxmk06wo...
ok!
extracting coords for video xdVpKRj-ZUk...
ok!
extracting coords for video lW13aMZAEHw...
ok!
extracting coords for video dT1sxYTvcs4...
ok!
extracting coords for video I6P5jjZSEIE...
ok!
extracting coords for video 3QbOHnydfjw...
ok!
extracting coords for video gagIBqOg3M0...
ok!


**Obtendo os detalhes dos vídeos pesquisados**

Agora que obtemos todas as ids possíveis dos vídeos relacionados à nossa seed em duas iterações, vamos resgatar os detalhes de cada um, como visualizações e quantidade de comentários, para armazenarmos também em uma planilha, que será a planilha referente aos nós da nossa rede: 